<a href="https://colab.research.google.com/github/HayyanWasi/openai-agent-learning/blob/main/08_streaming.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai google-generativeai openai-agents python-dotenv

In [ ]:
import nest_asyncio
nest_asyncio.apply()

# Basic Agent Setup

In [ ]:
import os
from agents import Agent, Runner, AsyncOpenAI, set_default_openai_client, set_tracing_disabled, OpenAIChatCompletionsModel
from google.colab import userdata
from pydantic import BaseModel
gemini_api_key = userdata.get('GEMINI_API_KEY')
set_tracing_disabled(True)

external_client = AsyncOpenAI(
    api_key=gemini_api_key,
    base_url="https://generativelanguage.googleapis.com/v1beta/",
)

model = OpenAIChatCompletionsModel(
    model="models/gemini-2.0-flash",
    openai_client=external_client
)



# Streaming
streaming allow user to watch agents execution useful for showing progress
Runner.run_steamed() is used to initiate streaming


> Each event has a type (e.g., response.created, response.output_text.delta) and associated data.


1.   response.created: A type value for a RawResponsesStreamEvent that indicates the AI has started a new response. It’s like a signal that says, “I’m beginning to answer the request.”
2.   response.output_text.delta: A type value for a RawResponsesStreamEvent that indicates the AI is sending a small chunk of text (a “delta”) as part of its response. This is the actual text being generated,






> There are 3 events of streaming
1.   RawResponsesStreamEvent: which represents raw events directly from the LLM
2.   RunItemStreamEvent: Represents the completion of an item, such as a message, tool call, or tool output.
3.   AgentUpdatedStreamEvent: riggered when the current agent changes, typically due to a handoff to another agent.



In [ ]:
import asyncio
from openai.types.responses import ResponseTextDeltaEvent #type of event of stream that represents small chunk of text generated by LLM
from agents import Agent, Runner

async def main():
    agent = Agent(
        name="Joker",
        instructions="You are a helpful assistant.",
        model=model,
    )

    result = Runner.run_streamed(agent, input="Please tell me 5 jokes.")
    async for event in result.stream_events():                            #instead of waiting for the entire response at once. we itterate over stream of events
        if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):#used to check program process right updates only in text chunks
            print(event.data.delta, end="", flush=False)

    print(result.final_output)

if __name__ == "__main__":
    asyncio.run(main())


Alright, here are 5 jokes for you:

1.  Why don't scientists trust atoms?
    Because they make up everything!

2.  What do you call a lazy kangaroo?
    Pouch potato!

3.  Why did the bicycle fall over?
    Because it was two tired!

4.  What musical instrument is found in the bathroom?
    A tuba toothpaste!

5.  Parallel lines have so much in common.
    It’s a shame they’ll never meet.

Hope you enjoyed those! Let me know if you'd like to hear more.
Alright, here are 5 jokes for you:

1.  Why don't scientists trust atoms?
    Because they make up everything!

2.  What do you call a lazy kangaroo?
    Pouch potato!

3.  Why did the bicycle fall over?
    Because it was two tired!

4.  What musical instrument is found in the bathroom?
    A tuba toothpaste!

5.  Parallel lines have so much in common.
    It’s a shame they’ll never meet.

Hope you enjoyed those! Let me know if you'd like to hear more.



In [ ]:
import asyncio
import random
from agents import Agent, ItemHelpers, Runner, function_tool #ItemHelpers helps handle specific types of events, like extracting text from a message output.

@function_tool
def how_many_jokes() -> int:
    return random.randint(1, 10)


async def main():
    agent = Agent(
        name="Joker",
        instructions="First call the `how_many_jokes` tool, then tell that many jokes.",
        tools=[how_many_jokes],
        model=model

    )

    result = Runner.run_streamed(
        agent,
        input="Hello",
    )
    print("=== Run starting ===")

    async for event in result.stream_events():
        if event.type == "raw_response_event":
            continue
        elif event.type == "agent_updated_stream_event":
            print(f"Agent updated: {event.new_agent.name}")
            continue

        elif event.type == "run_item_stream_event":
            if event.item.type == "tool_call_item":
                print("-- Tool was called")
            elif event.item.type == "tool_call_output_item":
                print(f"-- Tool output: {event.item.output}")
            elif event.item.type == "message_output_item":
                print(f"-- Message output:\n {ItemHelpers.text_message_output(event.item)}")
            else:
                pass

    print("=== Run complete ===")


if __name__ == "__main__":
    asyncio.run(main())

=== Run starting ===
Agent updated: Joker
-- Tool was called
-- Tool output: 10
-- Message output:
 I am programmed to tell 10 jokes.

=== Run complete ===
